In [20]:
import csv
import os
import sys
# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame, Row
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import desc
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, expr
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Word2Vec
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
import numpy 

from csv import reader

In [2]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

<Font size=5 color=red>Investigate the original dataset (obviously, it cannot be used). Take a look at https://stackoverflow.com/questions/13793529/r-error-invalid-type-list-for-variable to see how useless the Body column information could be!

The point here is that the body information consists mostly of codes and some weird patterns that are not useful for our purpose. The most important information here is the connection between the title of the questions and tags. So, I removed the Body column from the dataset.</Font>

In [3]:
try:
    spark = init_spark()

    filename1 = "./Train.csv"
    df2 = spark.read.option("multiLine", 'true').option("escape","\'").csv(filename1, header=True)
    print(df2.count())
    print(df2.show(10))    
except:
    pass

62819203
+--------------------+--------------------+--------------------+--------------------+
|                  Id|               Title|                Body|                Tags|
+--------------------+--------------------+--------------------+--------------------+
|                   1|How to check if a...|<p>I'd like to ch...|php image-process...|
|                   2|How can I prevent...|<p>In my favorite...|             firefox|
|                   3|R Error Invalid t...|"<p>I am import m...|                null|
|      expert_trai...|                null|                null|                null|
|      expert_data...|                null|                null|                null|
|      rf_model = ...| data=expert_data...|     importance=TRUE|      do.trace=100);|
|                   }|                null|                null|                null|
|       </code></pre>|                null|                null|                null|
|<p>Structure of t...|                null|  

<Font size=5 color=red>For removing the Body column, I read all the dataset once using Pandas library. After that, I removed the column and got an export to have a concrete file as our dataset. This part has been ommited from the notebook.</Font>

In [3]:
spark = init_spark()

filename = "./TrainWithoutBody.csv"
df1 = spark.read.option("multiLine", 'true').option("escape","\'").csv(filename, header=True)
df1 = df1.drop("_c0")
df1 = df1.dropna()

rddTags = df1.select("Tags").rdd

df1.count()

6017243

In [4]:
df1.show(5)

+---+--------------------+--------------------+
| Id|               Title|                Tags|
+---+--------------------+--------------------+
|  1|How to check if a...|php image-process...|
|  2|How can I prevent...|             firefox|
|  3|R Error Invalid t...|r matlab machine-...|
|  4|How do I replace ...|     c# url encoding|
|  5|How to modify who...|php api file-get-...|
+---+--------------------+--------------------+
only showing top 5 rows



<Font size = 5, color=green>Finding the 100 most used tags (one DT per each most used tag)

In [4]:
splittedTags = rddTags.filter(lambda r: r[0] != None).flatMap(lambda r: r[0].split(" ")).map(lambda r: (r, 1)).reduceByKey(lambda x, y: x + y)

splittedTags = splittedTags.sortBy(lambda r: r[1], False) #Sorted with number of usage (you can collect and see)

splittedTagsSorted = splittedTags.map(lambda r: r[0]) #Delete this line if you want to see number of times they have been used.

# df10 = anSorted.toDF()


mostUsedTags = splittedTagsSorted.collect()[0:50]



In [5]:
mostUsedTags

['c#',
 'java',
 'php',
 'javascript',
 'android',
 'jquery',
 'c++',
 'python',
 'iphone',
 'asp.net',
 'mysql',
 'html',
 '.net',
 'ios',
 'objective-c',
 'sql',
 'css',
 'linux',
 'ruby-on-rails',
 'windows',
 'c',
 'sql-server',
 'ruby',
 'wpf',
 'xml',
 'ajax',
 'database',
 'regex',
 'windows-7',
 'asp.net-mvc',
 'xcode',
 'django',
 'osx',
 'arrays',
 'vb.net',
 'eclipse',
 'json',
 'facebook',
 'ruby-on-rails-3',
 'ubuntu',
 'performance',
 'networking',
 'string',
 'multithreading',
 'winforms',
 'security',
 'asp.net-mvc-3',
 'visual-studio-2010',
 'bash',
 'homework']

In [99]:
print(rddTags.count())

6017243


In [9]:
rddTags.take(10)

[Row(Tags='php image-processing file-upload upload mime-types'),
 Row(Tags='firefox'),
 Row(Tags='r matlab machine-learning'),
 Row(Tags='c# url encoding'),
 Row(Tags='php api file-get-contents'),
 Row(Tags='proxy active-directory jmeter'),
 Row(Tags='core-plot'),
 Row(Tags='c# asp.net windows-phone-7'),
 Row(Tags='.net javascript code-generation'),
 Row(Tags='sql variables parameters procedure calls')]

<Font size=5.5, color="purpule">Here, I have cleaned the Tags column to only contain the most used tags. For example, I ommited the "upload" tag from first group of tags for the first question, because it's not a most used tag.

In [101]:
def replaceNoneWithString(x):
    if (x == None): return "None"
    else : return x

In [ ]:
rrr = rddTags.map(lambda r: r[0]).map(replaceNoneWithString).map(lambda r: r.split(" ")).map(lambda r: [ped for ped in r if ped in mostUsedTags])
cleanedTags = rrr.take(10)
cleanedTags

# Subject titles to TF-IDF

In [103]:
tokenizer = Tokenizer(inputCol="Title", outputCol="transformed_tfidf")
wordsData = tokenizer.transform(df1)

hashingTF = HashingTF(inputCol="transformed_tfidf", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

featurizedData.take(1)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [12]:
rescaledData.select("id", "tags", "features").show()
rescaledData.take(1)

+---+--------------------+--------------------+
| id|                tags|            features|
+---+--------------------+--------------------+
|  1|php image-process...|(20,[1,3,4,8,9,10...|
|  2|             firefox|(20,[1,2,3,7,12,1...|
|  3|r matlab machine-...|(20,[2,4,5,6,7,17...|
|  4|     c# url encoding|(20,[3,6,7,10,13,...|
|  5|php api file-get-...|(20,[3,5,8,13,15,...|
|  6|proxy active-dire...|(20,[0,3,4,7,13,1...|
|  7|           core-plot|(20,[3,6,7,8,10,1...|
|  8|c# asp.net window...|(20,[0,3,7,8,10,1...|
|  9|.net javascript c...|(20,[0,1,3,4],[1....|
| 10|sql variables par...|(20,[0,3,6,12,16,...|
| 11|.net obfuscation ...|(20,[0,3,6,8,11,1...|
| 12|algorithm languag...|(20,[0,1,5,19],[1...|
| 13|postfix migration...|(20,[1,3,8,18,19]...|
| 14|documentation lat...|(20,[10,13,16,17,...|
| 15|           windows-7|(20,[1,2,12,13,15...|
| 16|php url-routing c...|(20,[2,4,10,18],[...|
| 17|   r temporary-files|(20,[3,5,8,13,15,...|
| 18|         wpf binding|(20,[0,8,12,13

[Row(Id='1', Title='How to check if an uploaded file is an image without mime type?', Tags='php image-processing file-upload upload mime-types', transformed_tfidf=['how', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'without', 'mime', 'type?'], rawFeatures=SparseVector(20, {1: 1.0, 3: 3.0, 4: 1.0, 8: 2.0, 9: 1.0, 10: 2.0, 12: 2.0, 16: 1.0}), features=SparseVector(20, {1: 1.1357, 3: 1.8792, 4: 1.1753, 8: 1.4617, 9: 1.1974, 10: 2.0765, 12: 2.0979, 16: 0.9553}))]

# Subject titles to Word2Vec

In [104]:
tokenizer = Tokenizer(inputCol="Title", outputCol="tokenized_text")
tokenized_df = tokenizer.transform(df1)

word2Vec = Word2Vec(inputCol="tokenized_text", outputCol="features", vectorSize=100)
w2v_model = word2Vec.fit(tokenized_df)
w2v_data = w2v_model.transform(tokenized_df)


## Sample the Dataset

In [8]:
sampled_df = df1.sample(False, 0.0083, seed=42)

## Sampled Word2Vec

In [9]:
tokenizer_sampled = Tokenizer(inputCol="Title", outputCol="tokenized_text")
tokenized_df_sampled = tokenizer_sampled.transform(sampled_df)

word2Vec_sampled = Word2Vec(inputCol="tokenized_text", outputCol="features", vectorSize=100)
w2v_model_sampled = word2Vec_sampled.fit(tokenized_df_sampled)
w2v_data_sampled = w2v_model_sampled.transform(tokenized_df_sampled)

In [10]:
w2v_data_sampled.take(1)

[Row(Id='256', Title='How to figure out all colors in a gradient?', Tags='javascript html html5 colors mobile-safari', tokenized_text=['how', 'to', 'figure', 'out', 'all', 'colors', 'in', 'a', 'gradient?'], features=DenseVector([-0.0886, -0.0275, -0.0055, -0.1521, 0.0222, -0.0349, -0.0111, -0.0664, -0.007, -0.0018, -0.0511, 0.0199, 0.0044, 0.0184, 0.0305, -0.0008, 0.0784, 0.0005, 0.0343, 0.0095, -0.104, -0.0573, 0.0101, -0.0103, 0.0158, -0.0487, -0.0471, 0.0364, -0.0332, -0.0173, -0.0651, -0.0013, 0.0944, 0.0042, -0.0048, 0.0996, 0.0109, -0.0997, -0.037, 0.0819, 0.0627, -0.0586, 0.0279, -0.008, 0.0465, -0.0168, -0.0181, 0.0759, -0.0275, 0.0039, 0.0801, 0.0612, 0.0232, -0.0785, -0.0415, 0.0093, 0.0227, 0.0236, 0.1114, -0.1044, 0.0061, -0.049, 0.1084, 0.0929, -0.0442, -0.0572, 0.0531, 0.0711, 0.0156, 0.0831, 0.0194, 0.0806, -0.0228, 0.0172, -0.0613, 0.0158, -0.1076, -0.0305, 0.0112, 0.064, -0.035, -0.0226, -0.0497, -0.0714, -0.0668, -0.0185, 0.0443, 0.118, 0.1433, -0.0684, -0.0112, -0.00

# Logistic Regression 

### Utility Functions and Constants

In [11]:
def my_function(r):
    tags = r.labels
    labels = []
    for t in mostUsedTags:
        if t in tags:
            labels.append(1)
        else:
            labels.append(0)
    return (r.Id, r.Title, r.features, labels)

### Get Result Array

In [12]:
target_tags = w2v_data_sampled.withColumn('labels', split(col("Tags")," "))
w2v_with_targets = target_tags.rdd.map(lambda r: my_function(r))
train_cols = ["Id","Title", "features", "labels"]
train_df = w2v_with_targets.toDF(train_cols)

+----+--------------------+--------------------+--------------------+
|  Id|               Title|            features|              labels|
+----+--------------------+--------------------+--------------------+
| 256|How to figure out...|[-0.0886373691674...|[0, 0, 0, 1, 0, 0...|
| 314|Save loop deletin...|[-0.0853397778959...|[0, 0, 0, 0, 0, 0...|
| 396|Magento how to mo...|[-0.0964386757049...|[0, 0, 1, 0, 0, 0...|
| 713|Installing Numpy ...|[0.08249647377265...|[0, 0, 0, 0, 0, 0...|
|1200|Return values fro...|[-0.0844753608107...|[0, 1, 0, 0, 0, 0...|
|1209|Emulating varianc...|[-0.0069562578573...|[1, 0, 0, 0, 0, 0...|
|1345|android network a...|[0.03282853712638...|[0, 0, 0, 0, 1, 0...|
|1406|Significance Tests R|[-0.0210384248445...|[0, 0, 0, 0, 0, 0...|
|1440|Calling a functio...|[-0.0376302030765...|[0, 0, 0, 0, 0, 0...|
|1449|jQuery running fu...|[-0.0372561349843...|[0, 0, 0, 0, 0, 1...|
|1523|in grails ,how us...|[-0.0155541035087...|[0, 0, 0, 0, 0, 0...|
|1644|Why do my blac

### Seperate Labels into Columns

In [45]:
arr_size = 50
train_df = train_df.select(['Id']+['Title']+['features']+['labels']+[expr('labels[' + str(x) + ']') for x in range(0, arr_size)])
colnames = ['Id']+['Title']+['features']+['labels'] + [str(mostUsedTags[i]) for i in range(0, arr_size)] 
train_df = df.toDF(*colnames)
train_df.take(1)

NameError: name 'new_colnames' is not defined